In [2]:
import pandas as pd
import numpy as np

In [3]:
train = pd.read_csv('data/ybigta_sdss_train.csv', index_col=0)
test = pd.read_csv('data/ybigta_sdss_test.csv', index_col=0)
sample_submission = pd.read_csv('data/ybigta_sdss_sample_submission.csv', index_col=0)

In [4]:
column_number = {}
for i, column in enumerate(sample_submission.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

train['type_num'] = train['type'].apply(lambda x: to_number(x, column_number))

In [5]:
#Robust scaling

from sklearn.preprocessing import RobustScaler

columns = train.columns[2:22]
rb_scaler = RobustScaler()
train_test = pd.concat((train.iloc[:, 2:22], test.iloc[:, 1:]), axis=0)
train_test = pd.DataFrame(rb_scaler.fit_transform(train_test), columns=columns, 
                          index=(list(train.index) + list(test.index)))

train_scaled = train_test.iloc[:len(train.index), :]
test_scaled = train_test.iloc[len(train.index):, :]
train = pd.concat((train.iloc[:, 0:2], train_scaled, train.iloc[:, 22]), axis=1)
test = pd.concat((test.iloc[:, 0], test_scaled), axis=1)

train.head

<bound method NDFrame.head of                        type  fiberID  psfMag_u  psfMag_g  psfMag_r  psfMag_i  \
id                                                                             
415567                  QSO      106 -0.435873 -0.293006 -0.061893 -0.112365   
733874                  QSO      492 -0.580260 -0.415342 -0.221442 -0.113627   
1009150                 QSO      388 -0.185381 -0.378678 -0.230361 -0.128357   
803041                  QSO      531  0.616548  0.437376  0.608051  0.845455   
432241                  QSO      180  0.929133  0.371992  0.167015  0.201739   
243530                  QSO       12  0.310785  0.097471  0.213110  0.350142   
1478802                 QSO      542 -0.217861 -0.178132 -0.034040  0.168839   
233471                  QSO      346 -0.351428 -0.040550  0.039564  0.061180   
18370                   QSO      398  0.001218 -0.062852  0.039538  0.067687   
1235393                 QSO      517  1.709263  0.822173  0.329555  0.060270   
1514081   

In [4]:
df.isnull().sum()

id            0
type          0
fiberID       0
psfMag_u      0
psfMag_g      0
psfMag_r      0
psfMag_i      0
psfMag_z      0
fiberMag_u    0
fiberMag_g    0
fiberMag_r    0
fiberMag_i    0
fiberMag_z    0
petroMag_u    0
petroMag_g    0
petroMag_r    0
petroMag_i    0
petroMag_z    0
modelMag_u    0
modelMag_g    0
modelMag_r    0
modelMag_i    0
modelMag_z    0
dtype: int64

type :
>천체의 종류. 종속변수.

fiberID :
>천체를 관측할 때 사용된 광섬유의 식별번호.

u/g/r/i/z :
> 관측 파장대. Ultraviolet(u) / Green(g) / Red(r) / Near Infrared(i) / Infrared(z)

어쩌구Mag :
>https://www.sdss.org/dr12/algorithms/magnitudes/#nmgy

fiberMag :
>The magnitude measured by the frames pipeline to simulate the flux that would fall into a 3″ fiber in typical seeing. 

psfMag :
> optimal measures of the fluxes of stars.

modelMag :
> optimal measure of the flux of a galaxy.


In [5]:
len(df.fiberID.unique())

1000

In [ ]:
df.groupby('type').size().sort_values(ascending=False)

QSO : 
> 퀘이사(QUASAR). QUASAR는 지구로부터 수십억 광년 떨어진 거리에 있는 활동성 은하의 핵으로서 태양보다 수 조배나 밝으며 고정되어 있으므로 우주공간상 위치측정의 기준이 된다. 0.05-7의 스펙트럼 적색편이를 보인다.

STAR :
> 항성. 천구에 붙박혀 있어서 별자리를 기준으로 거의 움직이지 않으며, 점같이 보이는 천체.
- STAR_RED_DWARF : 적색 왜성. 초거성 등에 비하여 어둡기 때문에 눈에 잘 띄지 않으나 수는 많아 근거리 항성(恒星)의 약 70%를 차지한다. 질량은 태양의 반 이하이다.
- STAR_WHITE_DWARF : 백색 왜성.
- STAR_BROWN_DWARF : 갈색 왜성. 행성보다는 크지만 항성보다는 질량이 작고, 가시광선 영역의 빛을 내지 못하는 천체. 즉 질량이 태양의 10분의 1보다 작고, 1000분의 1보다 큰 천체.
- STAR_SUB_DWARF : low-luminosity subdwarfs. 준왜성. 광도가 동일한 색인 보통의 왜성보다 약 0.5등급 어두운 왜성. 중원소 원자에 의한 청색에서 보라색 영역에 걸친 선스펙트럼 흡수가 적기 때문에 같은 질량, 같은 연령의 왜성에 비해 색이 푸르게 보이므로 어두운 쪽으로 벗어난 계열을 형성한다.
- STAR_BHB : Blue Horizontal-Branch stars. 청색거성 or 쌍성 추정
- STAR_CARBON : carbon stars, both dwarf and giant. 탄소별. 항성대기의 화학성분 중 탄소가 산소보다 많은 항성이다. 점근거성가지 단계의 탄소별은 표면온도가 낮지만(3000-2000 K), 반경이 매우커서 밝기가 태양의 5000-10000배에 해당하는 적색거성 또는 적색초거성이다.
- STAR_PN : the central stars of planetary nebulae. 행성상성운. 백색왜성 또는 백성왜성으로 진화하는 질량이 작은 별 주변에 형성된 팽창하는 고리 형태의 방출성운을 말한다.
- STAR_CATY_VAR : cataclysmic variables. 격변변광성. 밝기가 갑자기 밝아졌다가 원래 밝기로 돌아가는 변광성이다.

SERENDIPITY :
>An open category of targets used in SDSS-I and SDSS-II whose selection criteria explore different regions of 
parameter space. These include :
- objects lying outside the stellar locus in color space (SERENDIP_RED, SERENDIP_BLUE, SERENDIP_DISTANT)
- objects coincident with FIRST sources but fainter than the equivalent in quasar target selection; also not restricted to point sources (SERENDIP_FIRST)
- hand-selected targets (SERENDIP_MANUAL)
    
ROSAT :
> For typical values of the ratio of optical to X-ray flux of various classes of Galactic and extragalactic sources, there is an excellent match between the depth of SDSS in the optical and that of the ROSAT All Sky Survey (RASS; voges99) in X-rays. Objects detected in SDSS imaging data are positionally matched against X-ray sources from the RASS catalogs.
SDSS objects within the RASS positional error circles (commonly 10-20'' radius) are scrutinized further by target selection algorithms tuned to select likely optical counterparts to the X-ray sources. In decreasing priority order, spectra are obtained for SDSS/RASS coincidences that:
- are otherwise bright enough for follow-up spectroscopy (ROSAT_D)

SPECTROPHOTO_STD :
>spectrophotometric standard stars. 분광측광표준항성. 이미 광도가 알려져 분광측광의 비교측정으로 사용됨.

GALAXY :
> 외부 은하. 

SKY : 
    


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(20,20))
sns.heatmap(data = df.corr(), annot=True, fmt='.2f', linewidths=.5)

In [7]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
x_data = df.drop(columns=['id','type'])
vif["VIF Factor"] = [variance_inflation_factor(x_data.values, i) for i in range(x_data.shape[1])]
vif["features"] = x_data.columns
vif

,VIF Factor,features
0,1.111200,fiberID
1,5050.113990,psfMag_u
2,6.872919,psfMag_g
3,7.768348,psfMag_r
4,7.296348,psfMag_i
5,8.574168,psfMag_z
6,5032.722935,fiberMag_u
7,1.035935,fiberMag_g
8,2.968382,fiberMag_r
9,5.939947,fiberMag_i


In [8]:
vif = pd.DataFrame()
x_data = df.drop(columns=['id','type', 'psfMag_u','petroMag_u'])
vif["VIF Factor"] = [variance_inflation_factor(x_data.values, i) for i in range(x_data.shape[1])]
vif["features"] = x_data.columns
vif

,VIF Factor,features
0,1.079073,fiberID
1,6.411892,psfMag_g
2,6.841427,psfMag_r
3,7.066838,psfMag_i
4,5.909444,psfMag_z
5,1.000863,fiberMag_u
6,1.035578,fiberMag_g
7,2.153277,fiberMag_r
8,4.949025,fiberMag_i
9,6.744337,fiberMag_z


In [6]:
from sklearn.model_selection import train_test_split

train_x = train.drop(columns=['type', 'type_num'], axis=1)
train_y = train['type_num']
test_x = test

X_train, X_valid, y_train, y_valid = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

In [7]:
from sklearn.svm import SVC

svm = SVC(probability=True)
svm.fit(X_train, y_train)
svm_pred = svm.predict_proba(X_valid)

from sklearn import metrics
score = metrics.log_loss(y_valid, svm_pred)
score

C:\Users\space\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.7314559867617801

In [13]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators = 1000,
                            criterion = 'gini')
rf.fit(X_train, y_train)
rf_pred = rf.predict_proba(X_valid)

score = metrics.log_loss(y_valid, rf_pred)
score

0.4034741224425686

In [14]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators = 1000,
                            criterion = 'entropy')
rf.fit(X_train, y_train)
rf_pred = rf.predict_proba(X_valid)

score = metrics.log_loss(y_valid, rf_pred)
score

0.4064338122626459

In [15]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators = 2000,
                            criterion = 'gini')
rf.fit(X_train, y_train)
rf_pred = rf.predict_proba(X_valid)

score = metrics.log_loss(y_valid, rf_pred)
score

0.39356148862845614

In [7]:
from sklearn.ensemble import RandomForestClassifier

rf_3000 = RandomForestClassifier(n_estimators = 3000,
                            criterion = 'gini')
rf_3000.fit(X_train, y_train)
rf_3000_pred = rf.predict_proba(X_valid)

score = metrics.log_loss(y_valid, rf_3000_pred)
score

NameError: name 'rf' is not defined

In [8]:
rf_3000_pred = rf_3000.predict_proba(X_valid)
score = metrics.log_loss(y_valid, rf_3000_pred)
score

NameError: name 'metrics' is not defined

In [9]:
from sklearn import metrics

score = metrics.log_loss(y_valid, rf_3000_pred)
score

0.3911361489463834

In [163]:
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier(n_jobs=-1,
                      num_leaves=127,
                      n_estimators=1250,
                      learning_rate=0.001)
lgbm.fit(X_train, y_train)
lgbm_pred = lgbm.predict_proba(X_valid)

In [164]:
from sklearn import metrics
score = metrics.log_loss(y_valid, lgbm_pred)
score

0.821522412512289

In [168]:
y_pred = lgbm.predict_proba(test_x)

In [ ]:
#Catboost
import catboost as cb
cb_clf = cb.CatBoostClassifier(n_estimators = 2000,
                               learning_rate = 0.001)
cb_clf.fit(X_train,y_train)


0:	learn: 2.9349938	total: 287ms	remaining: 9m 33s
1:	learn: 2.9254508	total: 570ms	remaining: 9m 29s
2:	learn: 2.9164754	total: 849ms	remaining: 9m 24s
3:	learn: 2.9073964	total: 1.16s	remaining: 9m 37s
4:	learn: 2.8984996	total: 1.45s	remaining: 9m 39s
5:	learn: 2.8896511	total: 1.76s	remaining: 9m 45s
6:	learn: 2.8807972	total: 2.11s	remaining: 10m
7:	learn: 2.8721458	total: 2.47s	remaining: 10m 14s
8:	learn: 2.8635416	total: 2.77s	remaining: 10m 13s
9:	learn: 2.8550359	total: 3.1s	remaining: 10m 17s
10:	learn: 2.8465270	total: 3.39s	remaining: 10m 12s
11:	learn: 2.8382610	total: 3.68s	remaining: 10m 10s
12:	learn: 2.8301374	total: 3.97s	remaining: 10m 6s
13:	learn: 2.8219768	total: 4.29s	remaining: 10m 8s
14:	learn: 2.8139377	total: 4.58s	remaining: 10m 5s
15:	learn: 2.8062661	total: 4.9s	remaining: 10m 7s
16:	learn: 2.7984188	total: 5.18s	remaining: 10m 4s
17:	learn: 2.7906428	total: 5.47s	remaining: 10m 1s
18:	learn: 2.7828166	total: 5.77s	remaining: 10m 1s
19:	learn: 2.7751850	t

In [ ]:
cb_pred = cb_model.predict_proba(X_valid)

In [ ]:
from sklearn import metrics
score = metrics.log_loss(y_valid, cb_pred)
score

In [199]:
from xgboost import XGBClassifier

xgb = XGBClassifier(booster = 'gbtree', 
                    n_jobs = -1, 
                    objective = 'multi:softprob', 
                    eval_metric = 'mlogloss',
                    estimator = 1000,
                    )
xgb.fit(X_train,y_train)
xgb_pred = xgb.predict_proba(X_valid)
score = metrics.log_loss(y_valid, xgb_pred)
score

0.38302130295075365

In [10]:
y_pred =rf_3000.predict_proba(test_x)

In [11]:
submission = pd.DataFrame(data=y_pred, columns=sample_submission.columns, index=sample_submission.index)
submission.to_csv('submission.csv', index=True)